# Analyzing used car listings data

In this project I'm going to analyze data extracted from *eBay Kleinanzeigen*, a classifieds sections of the German eBay website.

This dataset contains 50,000 data points,the data dictionary can be found [here](https://data.world/data-society/used-cars-data). 
The objective of this project is to clean and analze the car listings included, let's see if we can find something interesting.


In [2]:
import pandas as pd
import numpy as np

In [3]:
autos=pd.read_csv('autos.csv',encoding='Latin-1')
autos.info()
autos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


As we can see, of the twenty columns this dataset contains four of them have null values: `vehicleType`, `gearbox`, `model`and `fuelType`. 
`price`, `lastSeen`, `dateCrawled` and `dateCreated` are saved as strings.

### Renaming columns

Next, we're going to reword some column names to make them more descriptive, change the names from camelcase to snakecase and make the data easier to work with.

In [4]:
autos.columns=['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'powerPS', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'nr_of_pictures', 'postal_code',
       'last_seen']
autos.head()

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,powerPS,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


### Initial data exploration

Let's explore the data to find areas where it needs to be cleaned.

In [5]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,powerPS,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-23 19:38:20,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


By inspecting the columns with two unique values, we can see that the columns `seller` and `offer_type` contain values that are almost all the same, so we're goint to drop them as they don't have useful information for the analysis.

In [6]:
autos["seller"].value_counts()

privat        49999
gewerblich        1
Name: seller, dtype: int64

In [7]:
autos["offer_type"].value_counts()

Angebot    49999
Gesuch         1
Name: offer_type, dtype: int64

The 'nr_of_pictures' column looks very different to the rest of the colums, let's tak a closer look:

In [8]:
autos["nr_of_pictures"].value_counts()

0    50000
Name: nr_of_pictures, dtype: int64

It doesn't seem to contain any useful information, so we are going to drop this column as well.

In [9]:
autos.drop(["seller","offer_type","nr_of_pictures"],axis=1)

,date_crawled,name,price,abtest,vehicle_type,registration_year,gearbox,powerPS,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,39218,2016-04-01 14:38:50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,2016-03-27 14:38:19,Audi_Q5_3.0_TDI_qu._S_tr.__Navi__Panorama__Xenon,"$24,900",control,limousine,2011,automatik,239,q5,"100,000km",1,diesel,audi,nein,2016-03-27 00:00:00,82131,2016-04-01 13:47:40
49996,2016-03-28 10:50:25,Opel_Astra_F_Cabrio_Bertone_Edition___TÜV_neu+...,"$1,980",control,cabrio,1996,manuell,75,astra,"150,000km",5,benzin,opel,nein,2016-03-28 00:00:00,44807,2016-04-02 14:18:02
49997,2016-04-02 14:44:48,Fiat_500_C_1.2_Dualogic_Lounge,"$13,200",test,cabrio,2014,automatik,69,500,"5,000km",11,benzin,fiat,nein,2016-04-02 00:00:00,73430,2016-04-04 11:47:27
49998,2016-03-08 19:25:42,Audi_A3_2.0_TDI_Sportback_Ambition,"$22,900",control,kombi,2013,manuell,150,a3,"40,000km",11,diesel,audi,nein,2016-03-08 00:00:00,35683,2016-04-05 16:45:07


As we can see, there are a couple columns which contain numeric data stored as text, let's convert those to the appropiate format.

In [10]:
autos["price"]= (autos["price"]
                .str.replace("$","")
                .str.replace(",","")
                .astype(int))
autos["price"].head()

0    5000
1    8500
2    8990
3    4350
4    1350
Name: price, dtype: int64

In [11]:
autos["odometer"]=(autos["odometer"]
                  .str.replace("km","")
                  .str.replace(",","")
                  .astype(int))
autos["odometer"].head()

0    150000
1    150000
2     70000
3     70000
4    150000
Name: odometer, dtype: int64

In [12]:
autos.rename({"odometer":"odometer_km"},axis=1,inplace=True)

### Analyzing `price` and `odometer_km` columns

Now we're going to start looking for data that doesn't look right inside each column.

In [13]:
autos["price"].describe()

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64

In [14]:
autos["price"].value_counts().sort_index(ascending=True).head(10)

0     1421
1      156
2        3
3        1
5        2
8        1
9        1
10       7
11       2
12       3
Name: price, dtype: int64

In [15]:
autos["price"].value_counts().sort_index(ascending=False).head(20)

99999999    1
27322222    1
12345678    3
11111111    2
10000000    1
3890000     1
1300000     1
1234566     1
999999      2
999990      1
350000      1
345000      1
299000      1
295000      1
265000      1
259000      1
250000      1
220000      1
198000      1
197000      1
Name: price, dtype: int64

Low price values can be attributed to the fact that eBay is an auction site, however auctions start bids of \\$1 and seeing as the amount of zeroes represent 2\% of our dataset, we can remove them.
The most expensive car on this list is almost one billion dollars, we could limit the price of cars on $350,000, as it gradually builds up to that number and then makes a big jump.

In [16]:
autos=autos[autos["price"].between(0,350000)]
autos["price"].describe()

count     49986.000000
mean       5721.525167
std        8983.617820
min           0.000000
25%        1100.000000
50%        2950.000000
75%        7200.000000
max      350000.000000
Name: price, dtype: float64

Now let's look at the odotomer data:

In [17]:
autos["odometer_km"].describe()

count     49986.000000
mean     125736.506222
std       40038.133399
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

In [18]:
autos["odometer_km"].value_counts().sort_index(ascending=False)

150000    32416
125000     5169
100000     2168
90000      1757
80000      1436
70000      1230
60000      1164
50000      1025
40000       818
30000       789
20000       784
10000       264
5000        966
Name: odometer_km, dtype: int64

Most cars listed have high a lot of kilometers in them, as we can see from the rounded and uniform data of this field, users were most likely given the option to choose from a predetermined list of options.

### Analyzing date columns

The following columns include date information:

* `date_crawled`
* `last_seen`
* `ad_created`
* `registration_month`
* `registration_year`

These dates are a combination of dates added by the crawler when extracting the information and the columns take from the website.
Non-registration fields are stored as strings.

Let's understand how the data is represented on the columns formatted as strings:

In [19]:
autos[['date_crawled','last_seen','ad_created']].head()

,date_crawled,last_seen,ad_created
0,2016-03-26 17:47:46,2016-04-06 06:45:54,2016-03-26 00:00:00
1,2016-04-04 13:38:56,2016-04-06 14:45:08,2016-04-04 00:00:00
2,2016-03-26 18:57:24,2016-04-06 20:15:37,2016-03-26 00:00:00
3,2016-03-12 16:58:10,2016-03-15 03:16:28,2016-03-12 00:00:00
4,2016-04-01 14:38:50,2016-04-01 14:38:50,2016-04-01 00:00:00


We can extract only the date on a 'YYYY-MM-dd' format to see how these dates are distributed, by taking the first 10 characters of each column.

In [20]:
(autos['date_crawled']
     .str[:10]
     .value_counts(normalize=True,dropna=False)
     .sort_index())

2016-03-05    0.025387
2016-03-06    0.013944
2016-03-07    0.035970
2016-03-08    0.033269
2016-03-09    0.033209
2016-03-10    0.032129
2016-03-11    0.032489
2016-03-12    0.036770
2016-03-13    0.015564
2016-03-14    0.036630
2016-03-15    0.033990
2016-03-16    0.029508
2016-03-17    0.031509
2016-03-18    0.013064
2016-03-19    0.034910
2016-03-20    0.037831
2016-03-21    0.037490
2016-03-22    0.032909
2016-03-23    0.032389
2016-03-24    0.029108
2016-03-25    0.031749
2016-03-26    0.032489
2016-03-27    0.031049
2016-03-28    0.034850
2016-03-29    0.034150
2016-03-30    0.033629
2016-03-31    0.031909
2016-04-01    0.033809
2016-04-02    0.035410
2016-04-03    0.038691
2016-04-04    0.036490
2016-04-05    0.013104
2016-04-06    0.003181
2016-04-07    0.001420
Name: date_crawled, dtype: float64

In [21]:
(autos['date_crawled']
     .str[:10]
     .value_counts(normalize=True,dropna=False)
     .sort_values())

2016-04-07    0.001420
2016-04-06    0.003181
2016-03-18    0.013064
2016-04-05    0.013104
2016-03-06    0.013944
2016-03-13    0.015564
2016-03-05    0.025387
2016-03-24    0.029108
2016-03-16    0.029508
2016-03-27    0.031049
2016-03-17    0.031509
2016-03-25    0.031749
2016-03-31    0.031909
2016-03-10    0.032129
2016-03-23    0.032389
2016-03-26    0.032489
2016-03-11    0.032489
2016-03-22    0.032909
2016-03-09    0.033209
2016-03-08    0.033269
2016-03-30    0.033629
2016-04-01    0.033809
2016-03-15    0.033990
2016-03-29    0.034150
2016-03-28    0.034850
2016-03-19    0.034910
2016-04-02    0.035410
2016-03-07    0.035970
2016-04-04    0.036490
2016-03-14    0.036630
2016-03-12    0.036770
2016-03-21    0.037490
2016-03-20    0.037831
2016-04-03    0.038691
Name: date_crawled, dtype: float64

We can see that the data was extracted in the span of a month, from march 5th, 2016 to april 7th, 2016. 

In [22]:
(autos['ad_created']
     .str[:10]
     .value_counts(normalize=True,dropna=False)
     .sort_index())

2015-06-11    0.000020
2015-08-10    0.000020
2015-09-09    0.000020
2015-11-10    0.000020
2015-12-05    0.000020
                ...   
2016-04-03    0.038931
2016-04-04    0.036850
2016-04-05    0.011843
2016-04-06    0.003261
2016-04-07    0.001280
Name: ad_created, Length: 76, dtype: float64

The dates these ads were created on spans from june 11th, 2016 to april 7th, 2016, however most seem to have been created recently.

In [23]:
(autos['last_seen']
     .str[:10]
     .value_counts(normalize=True,dropna=False)
     .sort_values())

2016-03-05    0.001080
2016-03-06    0.004421
2016-03-07    0.005362
2016-03-18    0.007422
2016-03-08    0.007582
2016-03-13    0.008983
2016-03-09    0.009843
2016-03-10    0.010763
2016-03-11    0.012524
2016-03-14    0.012804
2016-03-19    0.015744
2016-03-15    0.015884
2016-03-27    0.016024
2016-03-16    0.016445
2016-03-26    0.016965
2016-03-23    0.018585
2016-03-25    0.019205
2016-03-24    0.019565
2016-03-20    0.020706
2016-03-21    0.020726
2016-03-28    0.020846
2016-03-22    0.021586
2016-03-29    0.022326
2016-04-01    0.023106
2016-03-12    0.023807
2016-03-31    0.023827
2016-04-04    0.024627
2016-03-30    0.024847
2016-04-02    0.024887
2016-04-03    0.025367
2016-03-17    0.027928
2016-04-05    0.124275
2016-04-07    0.130957
2016-04-06    0.220982
Name: last_seen, dtype: float64

These column can help us determine the percentage of cars that were sold since they were first crawled, as it is the last time the crawler saw it listed. However, the last three dates have an unsually high percentage, which is probably related to the date the last cralw was done and not the cars sold.

Next, let's see what the distribution of the registration years is:

In [24]:
autos["registration_year"].describe()

count    49986.000000
mean      2005.075721
std        105.727161
min       1000.000000
25%       1999.000000
50%       2003.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

As we can see, some of these data is incorrect, as there are registration years from 1,000 to 9,999.

Any car with a registration date after 2016, will be inaccurate because of the dates these ads were created on.
Determining the earliest valid year is a little bit more tricky, because some cars may be as old as to have been registered in the first decades of the 1900s.

Let's analyze what could be an appropiate lower bound:

In [25]:
autos["registration_year"].value_counts().sort_index().head(10)

1000    1
1001    1
1111    1
1500    1
1800    2
1910    9
1927    1
1929    1
1931    1
1934    2
Name: registration_year, dtype: int64

With this data, we can establish 1910 as our lower bound, with only six cars falling outside this bound.
Let's see the percentage of cars that fall outside the range between 1910-2016:

In [26]:
(~autos["registration_year"].between(1910,2016)).sum() / autos.shape[0]

0.03941103508982515

The percentage of cars that fall outside our range make up only 3.9% of our data, so we will remove these values.

In [27]:
autos=autos[autos["registration_year"].between(1910,2016)]
autos["registration_year"].value_counts(normalize=True).head(10)

2000    0.069852
2005    0.062792
1999    0.062438
2004    0.057002
2003    0.056794
2006    0.056377
2001    0.056273
2002    0.052753
1998    0.051087
2007    0.047984
Name: registration_year, dtype: float64

It seems like most cars were registered in the last 2 decades.

### Analysis by brand

We may be interested on doing an analysis of the price of the cars by brand, we'll analyze our data by looking at the top 10 brands with the most ads.

In [53]:
autos_brands=autos['brand'].value_counts().head(10)
print(autos_brands)

volkswagen       10185
bmw               5283
opel              5194
mercedes_benz     4579
audi              4149
ford              3350
renault           2274
peugeot           1418
fiat              1242
seat               873
Name: brand, dtype: int64


Volkswagen has the most ads of any other brand, having almost double the ads than the second place BMW.
German manufacturers also represent for out of the top five brands. 
Let's calculate the average price for each brand:

In [56]:
autos_brands=autos['brand'].value_counts().head(10).index

top10_prices={}

for brand in autos_brands:
    brand_only=autos[autos['brand']==brand]
    mean_price=brand_only['price'].mean()
    top10_prices[brand]=int(mean_price)
    
print(top10_prices)

{'volkswagen': 5231, 'bmw': 8102, 'opel': 2876, 'mercedes_benz': 8485, 'audi': 9093, 'ford': 3652, 'renault': 2395, 'peugeot': 3039, 'fiat': 2711, 'seat': 4296}


Of our top 20 brands we can see that:
* Audi, Mercedes Benz and BMW are the most expensive
* Renault, Fiat and Peugeot are the most accessible
* Volkswagen is a middle point between both, which may explain why it's the most popular based on ad listings.

### Analyzing car mileage

Now we're going to calculate the average mileage to understand if there's a link with the mean price.

In [64]:
top10_miles={}

for brand in autos_brands:
    brand_only=autos[autos['brand']==brand]
    avg_miles=brand_only['odometer_km'].mean()
    top10_miles[brand]=int(avg_miles)
    
price_series=pd.Series(top10_prices).sort_values(ascending=False)
miles_series=pd.Series(top10_miles).sort_values(ascending=False)


brand_info=pd.DataFrame(price_series,
                columns=['mean_price'])



brand_info['mean_miles']=miles_series

print(brand_info)

               mean_price  mean_miles
audi                 9093      129287
mercedes_benz        8485      130856
bmw                  8102      132431
volkswagen           5231      128724
seat                 4296      121563
ford                 3652      124068
peugeot              3039      127136
opel                 2876      129223
fiat                 2711      116553
renault              2395      128183


There doesn't seem to be a correlation between average price and average miles, as the mile information remains constant for the most part and the prices have a wider variation.